## Atividade

Realizar 3 consultas levando em conta as tabelas Clientes, Vendas, Itens Venda, Produtos, Vendedores:

- 1. Criar consulta mostrando nome, estado e status dos clientes

- 2. Criar consulta mostrando apenas os clientes Platinum e Gold

- 3. Demonstrar o total de Vendas de cada status

In [23]:
# Carregando libs e iniciando Session
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[1]").appName("Testing").getOrCreate()

### Consulta 1

In [24]:
# Importando tabela clientes
clientes = spark.read.format('parquet').load('arquivos/Atividades/Clientes.parquet')

In [25]:
# Conferindo a tabela
clientes.show(5)

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
+---------+--------------------+------+------+--------+
only showing top 5 rows



In [26]:
# Consulta
clientes.select('Cliente', 'Estado', 'Status').withColumnRenamed('Cliente','Nome').show()

+--------------------+------+--------+
|                Nome|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



### Consulta 2

In [27]:
# Consulta
clientes.filter((func.col('Status')=='Platinum')|(func.col('Status')=='Gold')).show()

+---------+-------------------+------+------+--------+
|ClienteID|            Cliente|Estado|Genero|  Status|
+---------+-------------------+------+------+--------+
|        4|   Adriana Guedelha|    RO|     F|Platinum|
|       10|   Alberto Monsanto|    RN|     M|    Gold|
|       28|      Anna Carvajal|    RS|     F|    Gold|
|       49|      Bento Quintão|    SP|     M|    Gold|
|       68|      Carminda Dias|    AM|     F|    Gold|
|       83|      Cláudio Jorge|    TO|     M|    Gold|
|      121|    Dionísio Saltão|    PR|     M|    Gold|
|      166|   Firmino Meireles|    AM|     M|    Gold|
|      170|      Flor Vilanova|    CE|     M|Platinum|
|      220|Honorina Villaverde|    PE|     F|    Gold|
|      230|    Ibijara Botelho|    RR|     F|Platinum|
|      237|  Iracema Rodríguez|    BA|     F|    Gold|
|      247|         Joana Ataí|    GO|     F|Platinum|
+---------+-------------------+------+------+--------+



### Consulta 3

In [28]:
# Importando tabela vendas
vendas = spark.read.format('parquet').load('arquivos/Atividades/Vendas.parquet')

In [29]:
# Conferindo a tabela
vendas.show(5)

+--------+----------+---------+--------+--------+
|VendasID|VendedorID|ClienteID|    Data|   Total|
+--------+----------+---------+--------+--------+
|       1|         1|       91|1/1/2019|  8053.6|
|       2|         6|      185|1/1/2020|   150.4|
|       3|         7|       31|2/1/2020|  6087.0|
|       4|         5|       31|2/1/2019| 13828.6|
|       5|         5|       31|3/1/2018|26096.66|
+--------+----------+---------+--------+--------+
only showing top 5 rows



In [33]:
# Checando o schema
vendas.schema

StructType([StructField('VendasID', LongType(), True), StructField('VendedorID', LongType(), True), StructField('ClienteID', LongType(), True), StructField('Data', StringType(), True), StructField('Total', DoubleType(), True)])

In [70]:
# Criando tabela de vendas por cliente
vendas_p_cliente = vendas.groupby('ClienteID').agg(sum('Total')).withColumnRenamed('ClienteID','IDCliente').withColumnRenamed('sum(Total)','Total')
vendas_p_cliente.show(5)

+---------+--------+
|IDCliente|   Total|
+---------+--------+
|      191|72001.75|
|       65| 2511.75|
|      243|   220.4|
|       54|12439.32|
|       19|  2656.1|
+---------+--------+
only showing top 5 rows



In [73]:
# Consulta
clientes.join(vendas_p_cliente ,clientes.ClienteID == vendas_p_cliente.IDCliente, "left").groupBy('Status').agg(sum('Total')).orderBy('sum(Total)').show()

+--------+-----------------+
|  Status|       sum(Total)|
+--------+-----------------+
|Platinum|         12584.68|
|    Gold|         27286.69|
|  Silver|3014291.359999999|
+--------+-----------------+

